In [3]:
# Standard Data Science Toolkit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# Statistics
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Machine Learning
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv('vgsales_clean.csv')
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


Before jumping into advanced techniques, let's establish a baseline model using Statmodels OLS regression.

Because this only takes in numeric values let's:
* Drop unneccessary columns like rank and name
* Encode Platform, genre, publisher

We'll predict on Global Sales and drop the others.

In [5]:
df = df.drop(columns=['Rank', 'Name', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'])
df.head()

,Platform,Year,Genre,Publisher,Global_Sales
0,Wii,2006,Sports,Nintendo,82.74
1,NES,1985,Platform,Nintendo,40.24
2,Wii,2008,Racing,Nintendo,35.82
3,Wii,2009,Sports,Nintendo,33.00
4,GB,1996,Role-Playing,Nintendo,31.37


In [6]:
cat_cols = ['Platform', 'Genre', 'Publisher']
num_cols = ['Year']

In [7]:
cat_df = df[cat_cols]
dummies = pd.get_dummies(cat_df, drop_first=True, dtype=int)
dummies

,Platform_3DO,Platform_3DS,Platform_DC,Platform_DS,Platform_GB,Platform_GBA,Platform_GC,Platform_GEN,Platform_GG,Platform_N64,...,Publisher_Zushi Games,Publisher_bitComposer Games,Publisher_dramatic create,Publisher_fonfun,Publisher_iWin,Publisher_id Software,Publisher_imageepoch Inc.,Publisher_inXile Entertainment,"Publisher_mixi, Inc",Publisher_responDESIGN
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16296,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16297,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16298,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16299,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
numeric = df[num_cols]
scaler = StandardScaler()

scaled_numeric = scaler.fit_transform(numeric)
scaled_numeric_df = pd.DataFrame(scaled_numeric, columns=numeric.columns)

In [9]:
X = pd.merge(dummies, scaled_numeric_df, left_index=True, right_index=True)
y = df['Global_Sales']

In [10]:
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Global_Sales   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     4.139
Date:                Fri, 14 Feb 2025   Prob (F-statistic):          6.36e-210
Time:                        16:35:51   Log-Likelihood:                -29227.
No. Observations:               16301   AIC:                         5.969e+04
Df Residuals:                   15683   BIC:                         6.445e+04
Df Model:                         617                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
const                                               -0.4360      0.886     -0.492      0.623      -2.172       1.300
Platform_3DO                                         0.1585      1.166      0.136      0.892      -2.128       2.445
Platform_3DS                                         0.7381      0.259      2.855      0.004       0.231       1.245
Platform_DC                                          0.3208      0.300      1.070      0.284      -0.267       0.908
Platform_DS                                          0.6741      0.234      2.880      0.004       0.215       1.133
Platform_GB                                          1.1845      0.252      4.697      0.000       0.690       1.679
Platform_GBA                                         0.2562      0.222      1.154      0.249      -0.179       0.691
Platform_GC                                          0.2034      0.226      0.900      0.368      -0.240       0.646
Platform_GEN                                         0.6197      0.361      1.716      0.086      -0.088       1.327
Platform_GG                                         -0.5363      1.496     -0.359      0.720      -3.468       2.396
Platform_N64                                         0.2467      0.222      1.109      0.267      -0.189       0.683
Platform_NES                                         1.0345      0.240      4.304      0.000       0.563       1.506
Platform_NG                                          0.1987      0.617      0.322      0.748      -1.011       1.409
Platform_PC                                          0.5261      0.240      2.190      0.029       0.055       0.997
Platform_PCFX                                       -0.2425      1.926     -0.126      0.900      -4.018       3.533
Platform_PS                                          0.5249      0.209      2.517      0.012       0.116       0.934
Platform_PS2                                         0.7579      0.222      3.410      0.001       0.322       1.194
Platform_PS3                                         1.1035      0.245      4.512      0.000       0.624       1.583
Platform_PS4                                         1.4231      0.269      5.282      0.000       0.895       1.951
Platform_PSP                                         0.6641      0.237      2.802      0.005       0.199       1.129
Platform_PSV                                         0.8549      0.264      3.243      0.001       0.338       1.372
Platform_SAT                                         0.1137      0.235      0.485      0.628      -0.346       0.573
Platform_SCD                                         0.0508      0.638      0.080      0.937      -1.200       1.301
Platform_SNES                            

Together, the features (platform, genre, publisher, and year) are only accouting for about 14% of the variance in Global Sales. Let's see if we can increase that using more advanced techniques.

In [11]:
# Machine Learning Preprocessing and Scoring Metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import  mean_squared_error, r2_score

# Machine Learning Algorithms
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

For Machine Learning it's important to scale the training and test data separately. With that in mind, let's redefine our X data, split, then scale.

In [12]:
X = pd.merge(dummies, df[num_cols], left_index=True, right_index=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=42)

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Now let's choose some models to determine how they compare to our baseline.

In [14]:
linreg = LinearRegression()
lasso = Lasso()
ridge = Ridge()
dtr = DecisionTreeRegressor(random_state=42)
rf = RandomForestRegressor(random_state=42)
gbr = GradientBoostingRegressor(random_state=42)
xgbr = XGBRegressor(random_state=42)
svr = SVR()

models = [linreg, lasso, ridge, dtr, rf, gbr, xgbr, svr]

In [17]:
def test_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    r2_train = r2_score(y_train, y_train_pred)
    
    y_test_pred = model.predict(X_test)
    r2_test = r2_score(y_test, y_test_pred)

    return f'Train: {r2_train:.2f} / Test: {r2_test:.2f}'

In [18]:
for model in models:
    print(f"{model} {test_model(model, X_train, y_train, X_test, y_test)}")

LinearRegression() Train: 0.16 / Test: -1154396084008847681781760.00
Lasso() Train: 0.00 / Test: -0.00
Ridge() Train: 0.16 / Test: 0.09
DecisionTreeRegressor(random_state=42) Train: 0.71 / Test: -0.32
RandomForestRegressor(random_state=42) Train: 0.65 / Test: -0.07
GradientBoostingRegressor(random_state=42) Train: 0.25 / Test: 0.08
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators

None of these models perform particularly well, given these inputs. This means that more data is needed to accurately perform predictive analysis. Still, let's see if we can advance the SVR model a bit since it currently has the lowest difference between training and testing performance. We can target model improvement through Hyperparameter Tuning!

In [ ]:
param_grid = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 10],
    'epsilon': [0.1, 1],
}

In [ ]:
gs = GridSearchCV(svr, param_grid, cv=3).fit(X_train, y_train)

In [ ]:
gs.best_score_